<a href="https://colab.research.google.com/github/guizar/ai4er-2021/blob/main/AI4ER_Supervised_classification_in_Google_Earth_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI4ER Practical: Introduction to Google Earth Engine

### Getting started

Earth Engine is a cloud-based data processing platform tailored for geospatial analysis. It runs on Google's servers and includes a multi-petabyte data catalog consisting mostly of satellite imagery, including the entire Landsat archive (NASA) as well as complete archives of data from Sentinel-1 and Sentinel-2 missions (ESA). 

If you have not already done so, you will need to signup for an EE account to run this practical:
https://signup.earthengine.google.com/#!/


Reference materials:
https://developers.google.com/earth-engine/guides/getstarted


### Activating Earth Engine

Run the following chunk to grant this notebook access to Earth Engine using your Google account.

In [46]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=MMeng0DiutD2TTSlyhgmZGKa2Tz-bxjaM4l1XUS2oUk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjPM66z-68UMy4dUjk8xWGorWJ_aiEb6-r0y3UqtbW848whjHYt0g4

Successfully saved authorization token.


## Interactive mapping using folium

Folium is a python library based on [leaflet.js](https://leafletjs.com/) that you can use to make interactive maps. It suppoert common mapping web-based formats which makes it ideal for visualizing the data produced in Earth Engine. We will also define a function to helop us process Earth Engine tiles with folium to display these interactively:

In [87]:
import folium

# Source: 

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Supervised classification in Google Earth Engine
In this practical we will produce a land-cover map of an area of interest using a supervised classification method. We will do this by fitting models with a set of training data of land-use categories (e.g forest, grassland, cropland, urban, water) and use these models to classify Landsat images into land-use categories.

### Training labels
Training classes to fit the models can be manually added on the map from the Earth Engine console (more of that below) or by loading a FeatureCollection object. In the following chunk we’ll load a set of spatial points with pre-labelled land-use categories for the island of Sao Miguel in the Azores (samples taken from the [ESA WorldCover 2020 dataset](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100)). We’ll also produce a region object to delimit our area of interest.

In [88]:
samples  = ee.FeatureCollection("users/guizarcou/lc_azores")
region = ee.Geometry.Polygon(
        [[[-25.858751237212292, 37.909865379396216],
          [-25.858751237212292, 37.70316988303885],
          [-25.117860734282605, 37.70316988303885],
          [-25.117860734282605, 37.909865379396216]]])


### Training data: Landsat 8
The Landsat 8 satellite captures wavelengths in the visible, near-infrared and short wave infrared at 30 mts resolution, which are separately stored in nine spectral bands. Our land-use classification will be based on the spectral signatures of six of these bands.

To get the best results, images need to be as cloud-free as possible and corrected before passing them to a training model. In the next chunk we will load a collection of atmospherically corrected Landsat 8 images and run a function to select cloud-free pixels from images taken between 2015 and 2020.


In [89]:
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Function to mask from the Fmask band of Landsat 8 SR data.
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()

  # Get the pixel QA band.
  qa = image.select('pixel_qa')

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  # Return the masked image, scaled to [0, 1].
  return image.updateMask(mask).divide(10000)

# Map the function over one year of data and take the median.
image = l8sr.filterDate('2015-01-01', '2020-12-31').map(maskL8sr).median()

In [90]:
import folium
# Set visualization parameters
l8_vis_params = {
    'min': 0, 'max': 0.3,
    'bands': ['B4', 'B3', 'B2']}

# Arrange layers 
ee_tiles = [image]

# Arrange visualization parameters inside a list.
ee_vis_params = [l8_vis_params]

# Arrange layer names inside a list.
ee_tiles_names = ['L8']

# Create a new map.
lon, lat = -25.46,37.77
my_map = folium.Map(location=[lat, lon], zoom_start=12)

# Add layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(my_map)

# my_map

Using the labels loaded in the samples object, we will sample pixels from the cleaned Landsat 8 mosaic and use these to train the classification tool. We will also define an object to select the bands that will be used for the classification:


In [91]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

training = image.select(bands).sampleRegions(
  collection= samples,
  properties= ['class'],
  scale= 30
)

### Classification 
Machine learning tools available in Earth Engine for pixel-based analyses include ~20 types of supervised classification, regression and unsupervised clustering methods. Here we will use a Classification And Regression Tree (CART) model and a Random Forest model. We wil compare their outputs in terms of their overall accuracy scores and their predictions over the area of interest



In [92]:
classifier_cart = ee.Classifier.smileCart().train(
  features= training,
  classProperty= 'class',
  inputProperties= bands
)


classifier_rf = ee.Classifier.smileRandomForest(40).train(
  features= training,
  classProperty= 'class',
  inputProperties= bands
)

Now we can visualise the result on a map. For reference, the classes used on the training set correspond to the following land-use classes:

Class | Land-use
------|------------
10    | Trees
30    | Grassland
40    | Cropland
50    | Built-up
80    | Water

In [93]:
classified_cart = image.select(bands).classify(classifier_cart)
classified_rf = image.select(bands).classify(classifier_rf)


# remap categories for plotting
classified_cart = classified_cart.remap([10,30,40,50,80],[1,2,3,4,5])
classified_rf = classified_rf.remap([10,30,40,50,80],[1,2,3,4,5])

# Set visualization parameters
pred_vis_params = {"palette": ["006400","ffff4c","f096ff","fa0000","0064c8"]}

# Arrange layers 
ee_tiles = [image,classified_cart,classified_rf]

# Arrange visualization parameters inside a list.
ee_vis_params = [l8_vis_params,pred_vis_params,pred_vis_params]

# Arrange layer names inside a list.
ee_tiles_names = ['Landsat 8','CART','Random Forests']

# Create a new map.
lon, lat = -25.46,37.77
my_map = folium.Map(location=[lat, lon], zoom_start=12)

# Add layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(my_map)

my_map

### Accuracy Assessments
We will now assess the accuracy of our predictions. The first step is to partition the set of known values into training and testing sets. Reusing the classification training set, add a column of random numbers used to partition the known data where about 70% of the data will be used for training and 30% for testing:


In [94]:
withRandom = training.randomColumn()

# Approximately 70% of our training data
trainingPartition = withRandom.filter(ee.Filter.lt('random', 0.7));
# Approximately 30% of our training data
testingPartition = withRandom.filter(ee.Filter.gte('random', 0.7));

# Trained with 70% of our data.
trainedClassifier_cart = ee.Classifier.smileCart().train(
  features=trainingPartition,
  classProperty='class',
  inputProperties=bands
)

# Now with RF
trainedClassifier_rf = ee.Classifier.smileRandomForest(40).train(
  features=training,
  classProperty='class',
  inputProperties=bands
)

Train the classifier with the `trainingSet` and examine the reported overall accuracy. These scores can be also broken down into producer's and consumer's accuracy scores. In the chunk below we'll produce accuracy scores for the CART and RF models:

In [95]:
test_cart = testingPartition.classify(trainedClassifier_cart)
confusionMatrix_cart = test_cart.errorMatrix('class', 'classification')

test_rf = testingPartition.classify(trainedClassifier_rf)
confusionMatrix_rf = test_rf.errorMatrix('class', 'classification')

print('CART Accuracy score:')
confusionMatrix_cart.accuracy().getInfo()
# print('Producer’s Accuracy:')
# confusionMatrix.producersAccuracy().getInfo()
# print('Consumer’s Accuracy:')
# confusionMatrix.consumersAccuracy().getInfo()



CART Accuracy score:


0.7074175824175825

In [96]:
print('RF Accuracy score:')
confusionMatrix_rf.accuracy().getInfo()

RF Accuracy score:


0.9917582417582418

We can also visualise the difference between the CART and RF predictions



In [97]:
# get the differences between CART and RF
difference = classified_cart.subtract(classified_rf)

# Set visualization parameters
dif_vis_params = {"palette": ['#b35806','#e08214','#fdb863','#fee0b6','#f7f7f7','#d8daeb','#b2abd2','#8073ac','#542788']}

# Arrange layers 
ee_tiles = [image,classified,classified_rf,difference]

# Arrange visualization parameters inside a list.
ee_vis_params = [l8_vis_params,pred_vis_params,pred_vis_params,dif_vis_params]

# Arrange layer names inside a list.
ee_tiles_names = ['Landsat 8','CART','RF','Difference']

# Create a new map.
lon, lat = -25.46,37.77
my_map = folium.Map(location=[lat, lon], zoom_start=12)

# Add layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(my_map)

my_map

> What problems may there be in using overall accuracy to understand how well a land use classifier is working?

## Classification challenge
**Provide your own training dataset**

Using the geometry tools and the Landsat composite as a background, you can digitize training polygons and add a land cover class for the classifiers.

* Go to the Earth Engine  [console](https://code.earthengine.google.com/)  and look for an aerea to run the classification.

* Draw a polygon around a bare ground area, then  [configure the import](https://developers.google.com/earth-engine/playground#geometry-tools) .  Import as Feature, then click **+ New property**.  Name the new property ‘class’ and give it a value of 0.  The dialog should show **class**: 0.  Name the import 'urban’. When doing this you might find it helpful to use the *Satellite* basemap instead of *Map*

* 	**+ new layer** > Draw a polygon around vegetation > import as Feature > add a property > name it ‘class’ and give it a value of 1.  Name the import 'vegetation'.  

* **+ new layer** > Draw a polygon around water > import as Feature > add a property > again, name it ‘class’ and give it a value of 2.  Name the import 'water'.  

* Once you have your polygons, go to the top of the code editor and click the `import` button. This will display the features that you created manually. Copy everything between `ee.Feature()` and paste it into the code editor below. Before loading make sure to delete the semicolons `;` as well as the colour references (`/*....*/`)

![](https://i.imgur.com/f3koL9M.gif)

You should have three `Feature` objects named 'urban’, 'vegetation' and 'water'.  Merge them into one `FeatureCollection` 


In [98]:
# Paste the polygons coordinates below:
urban =  ee.Feature(ee.Geometry.Polygon(
            [[[114.16690221983436, 22.316122884862597],
              [114.17162290770057, 22.31727421229202],
              [114.17037836271766, 22.322832211118936],
              [114.16535726744178, 22.32255431643468]]]),
        {
          "class": 0,
          "system:index": "0"
        })
vegetation = ee.Feature(ee.Geometry.Polygon(
            [[[114.18828451021257, 22.356921536673156],
              [114.19596635683122, 22.360215725876717],
              [114.19824087007585, 22.363946400291407],
              [114.18772661073747, 22.36307327268377]]]),
        {
          "class": 1,
          "system:index": "0"
        })
water = ee.Feature(ee.Geometry.Polygon(
            [[[114.08557250362693, 22.30192948459409],
              [114.10754515987693, 22.293591155286244],
              [114.11106421810447, 22.30717046542427],
              [114.0959580169326, 22.309870287884163]]]),
        {
          "class": 2,
          "system:index": "0"
        })

lc_classes = ee.FeatureCollection([urban,vegetation,water])
lc_classes.getInfo()

{'columns': {'class': 'Integer', 'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [[[114.16690221983436,
       22.316122884862597],
      [114.17162290770057, 22.31727421229202],
      [114.17037836271766, 22.322832211118936],
      [114.16535726744178, 22.32255431643468],
      [114.16690221983436, 22.316122884862597]]],
    'type': 'Polygon'},
   'id': '0',
   'properties': {'class': 0},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[114.18828451021257, 22.356921536673156],
      [114.19596635683122, 22.360215725876717],
      [114.19824087007585, 22.363946400291407],
      [114.18772661073747, 22.36307327268377],
      [114.18828451021257, 22.356921536673156]]],
    'type': 'Polygon'},
   'id': '1',
   'properties': {'class': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[114.08557250362693, 22.30192948459409],
      [114.10754515987693, 22.293591155286244],
      [114.11106421810447, 22.30717046542427],
      [114.0959580169326, 22.3098

You can now generate training pixels and run the classifiers

In [99]:
training = image.select(bands).sampleRegions(
  collection= lc_classes,
  properties= ['class'],
  scale= 30
);

classifier = ee.Classifier.smileCart().train(
  features= training,
  classProperty= 'class',
  inputProperties= bands
)


**Plotting the model predictions**


As shown in the previous example, Earth Engine will attempt to make a prediction on the whole globe when we run the classification algorithm. We can constrain this to our area of interest by clipping the prediction image. We show how to do this in the snippet below. To run this make sure you first create a roi using the rectangle feature so as to delimit the are where the predictions will be made (example provided below): 


In [100]:
roi =  ee.Geometry.Polygon(
        [[[114.07665098634588, 22.377439895086095],
          [114.07665098634588, 22.284390372489472],
          [114.21535337892401, 22.284390372489472],
          [114.21535337892401, 22.377439895086095]]])

# get coordinates to center the map
lon=roi.centroid().getInfo()['coordinates'][0]
lat=roi.centroid().getInfo()['coordinates'][1]

classified = image.select(bands).classify(classifier).clip(roi)
# remap categories for plotting
classified = classified.remap([0,1,2],[1,2,3])


# Set visualization parameters
cart_vis_params = {"palette": ["red","green","blue",]}

# Arrange layers 
ee_tiles = [classified]

# Arrange visualization parameters inside a list.
ee_vis_params = [cart_vis_params]

# Arrange layer names inside a list.
ee_tiles_names = ['CART']


my_map = folium.Map(location=[lat, lon], zoom_start=14)

# Add layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    my_map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(my_map)

my_map


**Can you improve the accuracy of the classifier?**

- Other classifiers.  Try some of the other classifier types in Earth Engine to see if the result is better or different. Why might some predict more accurately than others ? Which ML methods would be best? See who can get the highest accuracy!
- Different (more) training data. Try adjusting the shape and/or size of your training polygons to have a more representative sample of your classes.
Add more predictors. Try adding spectral indices to the input variables (e.g. by selecting a different satellite collection)


**Acknowledgements**

This practical was developed with content adapted from [Classification in Earth Engine](https://developers.google.com/earth-engine/classification) and [An Intro to the Earth Engine Python API](https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/intro-to-python-api-guiattard/index.ipynb) 